In [3]:
!pip install --upgrade pandas

  Using cached pandas-0.25.3-cp35-cp35m-manylinux1_x86_64.whl (10.3 MB)
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2016.6.1
ERROR: Cannot uninstall 'pytz'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [13]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd                              
import matplotlib.pyplot as plt
import sys
import time
import math
#from datetime import datetime
#%matplotlib inline
#from matplotlib.pyplot import rcParams
#rcParams['figure.figsize'] = 10,6



In [18]:
data = pd.read_excel('./SABRE NCe Data.xlsx', sheet_name='SABRE 2018-2019 Data')
#Parse string to datetime type
#nce['Date Opened'] = nce['Date Opened'].dt.strftime('%Y-%m')
#nce['Date Opened'] = pd.to_datetime(nce['Date Opened'], infer_datetime_format = True, format='%Y%m')
#analysis = nce[['Date Opened','NCe']]
#print(analysis.head(5))
#df = analysis.groupby('Date Opened', as_index=False).count()
#df = analysis.groupby('Date Opened', as_index=False).count()
#print(df.head(5))
#df = df[12:24]
#indexedNce = df.set_index(['Date Opened'])
#print(indexedNce)



In [19]:
data.head()

,Year Wise Nce Count,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Year Wise Investigation Count,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,NCes on Commodity Code,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Customer Temperature Impact,(All),NaN,NaN,NaN,NaN,Customer Temperature Impact,(All),NaN,NaN,NaN,NaN,NaN,NaN,Year Wise RCCA Count,NaN
1,Symptom Detail,(All),NaN,NaN,NaN,NaN,Symptom Detail,(All),NaN,NaN,NaN,Customer Temperature Impact,(All),NaN,Row Labels,Count of RCCA
2,Impact,(All),NaN,NaN,NaN,NaN,Safety Impact,(All),NaN,NaN,NaN,Symptom Detail,(All),NaN,2018,35
3,Safety Impact,(All),NaN,NaN,NaN,NaN,Wafer Scrap Impact,(All),NaN,NaN,NaN,Impact,(All),NaN,Qtr1,10
4,Wafer Scrap Impact,(All),NaN,NaN,NaN,NaN,Customer Name,(All),NaN,NaN,NaN,Safety Impact,(All),NaN,Qtr2,7


In [ ]:
## Plot Graph

plt.xlabel('Date')
plt.ylabel('Number of NCe')
plt.plot(indexedNce)

In [ ]:
#Determining Rolling Statistics

rolmean = indexedNce.rolling(window = 2).mean()
rolstd = indexedNce.rolling(window = 2).std()
print(rolmean, rolstd)


In [ ]:
#Plot Rolling Statistics

orig = plt.plot(indexedNce, color = 'blue',label = 'Original')
mean = plt.plot(rolmean,color = 'red',label = 'Rolling Mean')
std = plt.plot(rolstd ,color = 'black',label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)


In [ ]:

#Performing Dickey-Fuller Test

from statsmodels.tsa.stattools import adfuller

dftest = adfuller(indexedNce['NCe'], autolag = 'AIC')
dfoutput = pd.Series(dftest[0:4], index = ['Test-Statistics', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value

print(dfoutput)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determining Rolling Statistics
    movingAvg = timeseries.rolling(window = 2).mean()
    movingStd = timeseries.rolling(window = 2).std()
    #print(rolmean, rolstd)


    #Plot Rolling Statistics
    orig = plt.plot(timeseries,color = 'blue',label = 'Original')
    mean = plt.plot(movingAvg,color = 'red',label = 'Rolling Mean')
    std = plt.plot(movingStd,color = 'black',label = 'Rolling Std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    #Performing Dickey-Fuller Test
    print('Results of Dickey-Fuller Test ...')
    dftest = adfuller(timeseries['NCe'], autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test-Statistics', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)


In [ ]:
datasetDiffShifting = indexedNce- indexedNce.shift()
plt.plot(datasetDiffShifting)

In [ ]:
datasetDiffShifting.dropna(inplace=True)
test_stationarity(datasetDiffShifting)

In [ ]:
# Components of Time Series

from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(indexedNce, freq=4)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(indexedNce, label = 'Original')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(trend, label = 'Trend')
plt.legend(loc='best')

plt.subplot(413)
plt.plot(seasonal, label = 'Seasonality')
plt.legend(loc='best')

plt.subplot(414)
plt.plot(residual, label = 'Residuals')
plt.legend(loc='best')
plt.tight_layout()


In [ ]:
decomposedLogData = residual
decomposedLogData.dropna(inplace=True)
test_stationarity(decomposedLogData)

In [ ]:
datasetDiffShifting

In [ ]:
#ACF and #PACF Graphs

from statsmodels.tsa.stattools import acf, pacf
datasetDiffShifting.dropna(inplace=True)
lag_acf = acf(datasetDiffShifting, nlags = 10)
lag_pacf = pacf(datasetDiffShifting, nlags = 10, method = 'ols')

#Plot ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetDiffShifting)), linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetDiffShifting)), linestyle='--',color='gray')
plt.title('Autocorrelation Function')


#Plot PACF

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetDiffShifting)), linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetDiffShifting)), linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#AR Model
model = ARIMA(indexedNce, order=(3,1,0))
results_AR = model.fit(disp=-1)
plt.plot(datasetDiffShifting)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %4f'% sum((results_AR.fittedvalues-datasetDiffShifting['NCe'])**2))
print('Plotting AR Model')
print(results_AR.aic)

In [ ]:
#MA model

model = ARIMA(indexedNce, order=(0,1,1))
results_MA = model.fit(disp=-1)
plt.plot(datasetDiffShifting)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %4f'% sum((results_MA.fittedvalues-datasetDiffShifting['NCe'])**2))
print('Plotting MA Model')
print(results_MA.aic)

In [ ]:
import itertools
p=d=q=range(0,10)
pdq = list(itertools.product(p,d,q))
pdq

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
start_time = time.time()
mRSS = sys.float_info.max
minParam = None
for param in pdq:
    try:
        model = ARIMA(indexedNce, order=param)
        results_ARIMA = model.fit(disp=-1)
        if not math.isnan(sum((results_ARIMA.fittedvalues-datasetDiffShifting['NCe'])**2)):
            print(param, results_ARIMA.aic, sum((results_ARIMA.fittedvalues-datasetDiffShifting['NCe'])**2))
        if mRSS > sum((results_ARIMA.fittedvalues-datasetDiffShifting['NCe'])**2):
            mRSS = sum((results_ARIMA.fittedvalues-datasetDiffShifting['NCe'])**2)
            minParam = param
    except:
        continue
print("Below are the best parameters:")
print(minParam,mRSS)
end_time = time.time()
print("Total time taken to get the best parameters (in seconds): ", int(end_time - start_time))


In [ ]:
#ARIMA Model
model = ARIMA(indexedNce, order=(6,1,1))   
results_ARIMA = model.fit(disp=-1)
plt.plot(datasetDiffShifting)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %4f'% sum((results_ARIMA.fittedvalues-datasetDiffShifting['NCe'])**2))
print('Plotting ARIMA Model')
print(results_MA.aic)

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print(predictions_ARIMA_diff)

In [ ]:
#Convert to Cumulative sum
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum)

In [ ]:
predictions_ARIMA_log = pd.Series(indexedNce['NCe'].ix[0], index=indexedNce.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA_log
plt.plot(indexedNce)
plt.plot(predictions_ARIMA_log)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA_log-indexedNce['NCe'])**2)/len(indexedNce)))


In [ ]:
results_ARIMA.plot_predict(1,24)

In [ ]:
x=results_ARIMA.forecast(steps=12)
a = x[0]
a